In [2]:
import numpy as np 
import pandas as pd
import feather
import plotly
import plotly.graph_objects as go
import gc

pd.options.display.max_rows = 100


import sys
sys.path.insert(0, "/home/jupyter/kaggle/energy/src")
import utility

In [5]:
train_df, _, _, _, _ = utility.read_data(utility.CREATED_DATA_DIR, train=True, test=False, weather_train=False, weather_test=False, building=False)

Reading Data...
Shape of train_df : (20216100, 16)


/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning:

.labels was deprecated in version 0.24.0. Use .codes instead.



In [6]:
train_df.columns

Index(['site_id', 'building_id', 'timestamp', 'meter', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed', 'meter_reading'],
      dtype='object')

In [7]:
# Get the data for site_id 0 only
train_df_site_0 = train_df[train_df.site_id == 0]

## Electrical Meter

### May 1 seems to be an outlier. Check if all the buildings have the peak

In [8]:
building_ids = train_df_site_0.building_id.unique()

# May 1 seems to be an outlier. Check if all the buildings have the peak
# Building 40, 45, 29 doesn't have
for id in building_ids:
    temp = train_df_site_0[train_df_site_0.building_id == id]
    meter_reading_sum = temp[(temp.timestamp >= np.datetime64('2016-05-01')) 
                             & (temp.timestamp < np.datetime64('2016-05-02')) 
                             & (temp.meter == 0)]['meter_reading'].sum()
    print(f'For building {id} meter_reading on 1 st May 2016 is {meter_reading_sum}')

For building 0 meter_reading on 1 st May 2016 is 448.0
For building 1 meter_reading on 1 st May 2016 is 317.79559326171875
For building 2 meter_reading on 1 st May 2016 is 40.0
For building 3 meter_reading on 1 st May 2016 is 937.0
For building 4 meter_reading on 1 st May 2016 is 3592.0
For building 5 meter_reading on 1 st May 2016 is 54.0
For building 6 meter_reading on 1 st May 2016 is 214.0
For building 7 meter_reading on 1 st May 2016 is 1160.0
For building 8 meter_reading on 1 st May 2016 is 749.0
For building 9 meter_reading on 1 st May 2016 is 223.0
For building 10 meter_reading on 1 st May 2016 is 3289.0
For building 11 meter_reading on 1 st May 2016 is 1052.0
For building 12 meter_reading on 1 st May 2016 is 356.0
For building 13 meter_reading on 1 st May 2016 is 742.0
For building 14 meter_reading on 1 st May 2016 is 872.0
For building 15 meter_reading on 1 st May 2016 is 571.0
For building 16 meter_reading on 1 st May 2016 is 2870.0
For building 17 meter_reading on 1 st May 

### Check if May 20 have zero reading for any the building?

In [108]:
building_ids = train_df_site_0.building_id.unique()
# Check if May 20 have zero reading for any the building?
# Building 29, 40, 45, 53 have it
for id in building_ids:
    temp = train_df_site_0[train_df_site_0.building_id == id]
    meter_reading_sum = temp[((temp.timestamp >= np.datetime64('2016-05-20')) 
                              & (temp.timestamp < np.datetime64('2016-05-21')) 
                              & (temp.meter == 0))]['meter_reading'].sum()

    if (meter_reading_sum <= 0):
        print(f'For building {id} meter_reading on 19th May 2016 is {meter_reading_sum}')

For building 40 meter_reading on 19th May 2016 is 0.0
For building 45 meter_reading on 19th May 2016 is 0.0
For building 53 meter_reading on 19th May 2016 is 0.0
For building 29 meter_reading on 19th May 2016 is 0.0


### For these three buildings on which date the meter reading started?

In [12]:
# Check building 40
# From June 3rd 11 AM onwards there is data
utility.get_non_zero_meter_reading_timestamp(train_df_site_0, 
                                     building_id=40, 
                                     start_time='2016-06-03 00:00:00', 
                                     stop_time='2016-06-04 00:00:00')

Timestamp('2016-06-03 11:00:00')

In [14]:
# Check building 45
# From June 30th 13 HRS onwards there is data
utility.get_non_zero_meter_reading_timestamp(train_df_site_0, 
                                     building_id=45, 
                                     start_time='2016-06-30 00:00:00', 
                                     stop_time='2016-07-01 00:00:00')

Timestamp('2016-06-30 13:00:00')

In [15]:
# Check building 29
# From Aug 10th 00 HRS onwards there is data
utility.get_non_zero_meter_reading_timestamp(train_df_site_0, 
                                     building_id=29, 
                                     start_time='2016-08-10 00:00:00', 
                                     stop_time='2016-08-11 00:00:00')

Timestamp('2016-08-10 00:00:00')

In [16]:
# TODO : What to do for building 53?
# Based on the visualization, don't do anything with the Building 53 data

In [18]:
# Test code : Check the meter reading for a particular building
# for a particular day. This is just to make sure if the above 
# non-zero meter readings are correct or not
train_df_site_0[(train_df_site_0.building_id == 29) 
                & (train_df_site_0.timestamp >= np.datetime64('2016-08-10 00:00:00')) 
                & (train_df_site_0.timestamp < np.datetime64('2016-08-11 00:00:00')) & (train_df_site_0.meter == 0)]

,site_id,building_id,timestamp,meter,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,meter_reading
12136743,0,29,2016-08-10 00:00:00,0,Education,113866,2006,99,25.000000,8.0,22.799999,-1.0,1018.500000,130.0,3.6,455.268005
12139004,0,29,2016-08-10 01:00:00,0,Education,113866,2006,99,23.900000,NaN,22.200001,3.0,1018.700012,150.0,4.1,410.218994
12141262,0,29,2016-08-10 02:00:00,0,Education,113866,2006,99,23.900000,NaN,22.799999,-1.0,1019.500000,170.0,5.7,367.217010
12143523,0,29,2016-08-10 03:00:00,0,Education,113866,2006,99,23.900000,NaN,22.200001,-1.0,1019.599976,150.0,3.6,375.066986
12145785,0,29,2016-08-10 04:00:00,0,Education,113866,2006,99,23.900000,NaN,22.799999,-1.0,1019.000000,120.0,1.5,376.773010
12148042,0,29,2016-08-10 05:00:00,0,Education,113866,2006,99,23.900000,NaN,22.799999,-1.0,1018.799988,220.0,2.6,374.384003
12150305,0,29,2016-08-10 06:00:00,0,Education,113866,2006,99,23.900000,8.0,22.799999,-1.0,1018.099976,180.0,2.6,413.972992
12152566,0,29,2016-08-10 07:00:00,0,Education,113866,2006,99,23.900000,NaN,22.799999,0.0,1018.299988,160.0,2.6,451.171997
12154828,0,29,2016-08-10 08:00:00,0,Education,113866,2006,99,23.900000,NaN,22.799999,0.0,1018.000000,170.0,1.5,534.103027
12157091,0,29,2016-08-10 09:00:00,0,Education,113866,2006,99,23.900000,NaN,22.799999,0.0,1018.000000,150.0,2.1,702.354980


In [128]:
# Test code : Check for a particular building
train_df_site_0[(train_df_site_0.building_id == 53) 
                & (train_df_site_0.timestamp >= np.datetime64('2016-07-01 00:00:00')) & (train_df_site_0.meter == 0)]

,site_id,building_id,timestamp,meter,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,meter_reading
8424928,0,53,2016-06-04 20:00:00,0,Office,87742,1971,99,33.900002,NaN,19.400000,0.0,1014.000000,140.0,3.1,383.325989
8427273,0,53,2016-06-04 21:00:00,0,Office,87742,1971,99,32.799999,NaN,21.700001,0.0,1013.500000,110.0,7.2,194.938995
8429617,0,53,2016-06-04 22:00:00,0,Office,87742,1971,99,32.200001,4.0,20.600000,0.0,1013.200012,110.0,6.7,0.000000
8431963,0,53,2016-06-04 23:00:00,0,Office,87742,1971,99,31.100000,4.0,20.600000,0.0,1013.200012,110.0,7.2,0.000000
8434310,0,53,2016-06-05 00:00:00,0,Office,87742,1971,99,29.400000,4.0,21.100000,0.0,1013.700012,110.0,7.2,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19243377,0,53,2016-12-14 20:00:00,0,Office,87742,1971,99,27.799999,NaN,16.100000,0.0,1016.400024,270.0,4.6,0.000000
19245742,0,53,2016-12-14 21:00:00,0,Office,87742,1971,99,27.200001,NaN,16.700001,0.0,1016.299988,250.0,3.6,0.000000
19248108,0,53,2016-12-14 22:00:00,0,Office,87742,1971,99,26.700001,4.0,16.700001,0.0,1016.200012,260.0,4.1,0.000000
19250474,0,53,2016-12-14 23:00:00,0,Office,87742,1971,99,23.900000,NaN,16.700001,0.0,1016.000000,230.0,2.1,0.000000


In [21]:
# On May 20th at what time, the meter reading started

# I already know that building 40, 45, 53 does not start at 20th May
# Building 29 - Aug 10th 00 HRS
# Building 40 - June 3rd 11 AM
# Building 45 - June 30th 13 HRS onwards
# Building 53 - Don't do anything

building_ids = list(train_df_site_0.building_id.unique())
building_ids.remove(29)
building_ids.remove(40)
building_ids.remove(45)
building_ids.remove(53)

# Let's assume for other buildings it starts at May 20th
# Print the time from which meter reading has non-zero values
for id_ in building_ids:
    stamp = utility.get_non_zero_meter_reading_timestamp(train_df_site_0, 
                                         building_id=id_, 
                                         start_time='2016-05-20 00:00:00',
                                         stop_time='2016-05-21 00:00:00')
    print(f' building id : {id_} : {stamp}')

 building id : 0 : 2016-05-20 18:00:00
 building id : 1 : 2016-05-20 10:00:00
 building id : 2 : 2016-05-20 18:00:00
 building id : 3 : 2016-05-20 18:00:00
 building id : 4 : 2016-05-20 18:00:00
 building id : 5 : 2016-05-20 18:00:00
 building id : 6 : 2016-05-20 18:00:00
 building id : 7 : 2016-05-20 18:00:00
 building id : 8 : 2016-05-20 18:00:00
 building id : 9 : 2016-05-20 18:00:00
 building id : 10 : 2016-05-20 18:00:00
 building id : 11 : 2016-05-20 18:00:00
 building id : 12 : 2016-05-20 18:00:00
 building id : 13 : 2016-05-20 18:00:00
 building id : 14 : 2016-05-20 18:00:00
 building id : 15 : 2016-05-20 18:00:00
 building id : 16 : 2016-05-20 18:00:00
 building id : 17 : 2016-05-20 18:00:00
 building id : 18 : 2016-05-20 18:00:00
 building id : 19 : 2016-05-20 18:00:00
 building id : 20 : 2016-05-20 18:00:00
 building id : 21 : 2016-05-20 18:00:00
 building id : 22 : 2016-05-20 18:00:00
 building id : 23 : 2016-05-20 18:00:00
 building id : 24 : 2016-05-20 18:00:00
 building 

## Chilled Water Meter

In [133]:
# Buildings which have chilled water meter
train_df_site_0[train_df_site_0.meter == 1].building_id.unique()

array([  7,   9,  13,  14,  15,  29,  30,  31,  43,  50,  55,  60,  75,
        76,  78,  82,  91,  92,  95,  96,  97, 103,  28,  98],
      dtype=uint64)

Look into 
60 - Outlier and initial zeros (Aligned with electricity)?
82 - Looks different?
92 - Looks different?
95 - Check the gap?
103 - outlier
98 - long gap?

In [ ]:
train_df_site_0[(train_df_site_0.building_id == 60) 
                & (train_df_site_0.timestamp >= np.datetime64('2016-08-10 00:00:00')) 
                & (train_df_site_0.timestamp < np.datetime64('2016-08-11 00:00:00')) & (train_df_site_0.meter == 0)].head(2)